In [1]:
import sys
import os
from modules.CARLA import carla

Appending path to carla module D:\Research\ecemosp\src\modules\CARLA
Appending path to carla module D:\Research\ecemosp\src\modules\CARLA


c:\Users\ignacy\.conda\envs\carla-211\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [29]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import joblib
import tensorflow as tf
from carla.recourse_methods import GrowingSpheres, Dice, ActionableRecourse, CEM, Face
from carla.models.catalog import MLModelCatalog

In [30]:
def display_results(model, factual, counterfactuals):
    pred = model.predict(factual)
    print('Factual class:', pred.round(3)) 
    print('Counterfactuals class:')
    for pred in model.predict(counterfactuals):
        print(pred.round(3))

In [31]:
from carla.data.catalog import CsvCatalog
import json

with open('../data/adult_constraints.json', 'r') as f:
    constraints = json.load(f)

continuous = constraints['continuous_features_nonsplit']
categorical = constraints['categorical_features_nonsplit']
immutable = constraints['non_actionable_features']
columns_order = constraints['features_order_nonsplit']
target_feature = constraints['target_feature']

if target_feature in columns_order:
    columns_order = columns_order.remove(target_feature)

raw = pd.read_csv("../data/adult_train.csv")

#TODO TEST WHY NOT STABLE NUMERICALLY

dataset = CsvCatalog(dataset=raw,
                    continuous=continuous,
                    categorical=categorical,
                    immutables=immutable,
                    target=target_feature)
factual = dataset.df_test.iloc[5:6]  
dataset.df.head(2)

,age,education.num,capital.gain,capital.loss,hours.per.week,...,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia
0,0.616438,0.800000,0.0,0.0,0.397959,...,0.0,0.0,1.0,0.0,0.0
1,0.301370,0.866667,0.0,0.0,0.551020,...,0.0,0.0,1.0,0.0,0.0


In [6]:
from models.tfmodel import TFModelAdult
model = TFModelAdult(dataset)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               11008     
                                                                 
 elu (ELU)                   (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 elu_1 (ELU)                 (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 elu_2 (ELU)                 (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 2

DICE FOR NN

In [ ]:


hyperparams = {"num": 20}
# load a recourse model and pass black box model
gs = Dice(model, hyperparams)

# generate counterfactual examples
counterfactuals = gs.get_counterfactuals(factual)
counterfactuals

In [ ]:
display_results(model, factual, counterfactuals)

CAUSAL RECOURSE

In [ ]:
# from carla.recourse_methods.catalog.causal_recourse import samplers, constraints
# hypers = {
#     'optimization_approach': "brute_force",
#     'num_samples': 10,
#     'scm': None,
#     "constraint_handle": constraints.point_constraint,
#     "sampler_handle": samplers.sample_true_m0,
# }


# gs = CausalRecourse(model, hypers)
# # generate counterfactual examples
# counterfactuals = gs.get_counterfactuals(dataset.df[:1])
# counterfactuals

GROWING SPHERES

In [ ]:
factual

In [17]:
gs = GrowingSpheres(model)
# generate counterfactual examples
counterfactuals = gs.get_counterfactuals(factual)
counterfactuals

,age,education.num,capital.gain,capital.loss,hours.per.week,...,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia
8398,0.304089,0.65383,0.048598,0.002075,0.310955,...,0.0,0.0,0.0,0.0,0.0


In [ ]:
display_results(model, factual, counterfactuals)

ACTIONABLE RECOURSE (AR)

In [20]:
hyperparams = {
    'fs_size': 100
}

ar = ActionableRecourse(model, hyperparams)
counterfactuals = ar.get_counterfactuals(factual)
counterfactuals

[INFO] Start generating LIME coefficients [model.py get_counterfactuals]
[INFO] Finished generating LIME coefficients [model.py get_counterfactuals]
obtained 100 items in 1.0 seconds


,age,education.num,capital.gain,capital.loss,hours.per.week,...,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia
0,0.342466,0.533333,0.0,0.0,0.397959,...,0.0,0.0,0.0,0.0,0.0


In [ ]:
display_results(model, factual, counterfactuals)

CONTRASTIVE EXPLANATIONS METHOD (CEM)

In [14]:
# hyperparams = {
#         "data_name": 'test1',
#         "batch_size": 1,
#         "kappa": 0.1,
#         "init_learning_rate": 0.01,
#         "binary_search_steps": 9,
#         "max_iterations": 100,
#         "initial_const": 10,
#         "beta": 0.9,
#         "gamma": 0.0,
#         "mode": "PN",
#         "num_classes": 2,
#         "ae_params": {"hidden_layer": [20, 10, 7], "train_ae": True, "epochs": 5},
# }


# sess = None

# cm = CEM(sess = sess, mlmodel=model, hyperparams=hyperparams)

# counterfactuals = cm.get_counterfactuals(factual)
# counterfactuals

Train on 22512 samples, validate on 9649 samples
Epoch 1/5
22512/22512 [==============================] - 1s 35us/sample - loss: 0.0594 - val_loss: 0.0286
Epoch 2/5
 3648/22512 [===>..........................] - ETA: 0s - loss: 0.0277

c:\Users\ignacy\.conda\envs\carla-211\lib\site-packages\keras\engine\training_v1.py:2333: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


22512/22512 [==============================] - 0s 18us/sample - loss: 0.0253 - val_loss: 0.0232
Epoch 3/5
22512/22512 [==============================] - 0s 20us/sample - loss: 0.0221 - val_loss: 0.0210
Epoch 4/5
22512/22512 [==============================] - 0s 20us/sample - loss: 0.0200 - val_loss: 0.0190
Epoch 5/5
22512/22512 [==============================] - 0s 20us/sample - loss: 0.0180 - val_loss: 0.0173


AttributeError: module 'tensorflow' has no attribute 'placeholder'

Feasible and Actionable Counterfactual Explanations (FACE)

In [28]:
hyperparams = {
    'mode': 'knn',
    'fraction': 0.1,
}
fc = Face(model, hyperparams)

counterfactuals = fc.get_counterfactuals(factual)
counterfactuals.to_numpy()


array([[0.32876712, 0.53333333, 0.02885029, 0.        , 0.39795918,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [ ]:
display_results(model, factual, counterfactuals)